In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
from sklearn.pipeline import FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import Perceptron
from sklearn.svm import LinearSVC
import re
from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.neural_network import MLPClassifier

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
data_path1 = '../dataset/Tweets-airline-sentiment.csv'
#data_path2 = '../dataset/labeledTrainData_head12000.tsv'

In [3]:
data = pd.read_csv(data_path1)

In [4]:
#data.head()
label = data['airline_sentiment']

In [5]:
text = data['text']

In [6]:
label_tags = label.unique()
print(label_tags)

['neutral' 'positive' 'negative']


In [7]:
#replace text label with one-hot-labels


In [8]:
new_label = []
for l in label:
    if l == label_tags[0]:
        #new_label.append(np.array([0,0,1]))
        new_label.append(0)
    elif l == label_tags[1]:
        #new_label.append(np.array([0,1,0]))
        new_label.append(1)
    else:
        #new_label.append(np.array([1,0,0]))
        new_label.append(2)

In [9]:
new_label = np.asarray(new_label)

In [10]:
#get rid of '@airline_company_name
new_text = []

In [11]:
for t in text:
    t = re.sub('[0-9]', '', t)
    new_text.append(re.sub('^@\w+ *','', t))

In [12]:
#new_text

In [13]:
new_text = np.asarray(new_text)

In [14]:
new_text.shape, new_label.shape

((14640,), (14640,))

# Split

In [23]:
def split(x, y, n_split, test_train_split = 0.2):
# x: traning test, in array
#y: label
#n_split: number of groups divided from x and y
    if x.shape[0] != y.shape[0]:
        raise('samples and their labels\'number mismatched')
    else:
        instance_num = x.shape[0]
        train_num = int(instance_num*(1-test_train_split))
        permulated_arr = np.random.permutation(instance_num)
        train_arr, test_arr = permulated_arr[:train_num], permulated_arr[train_num:]
        train_each_num = int(train_num/n_split)
        test_each_num = int((instance_num-train_num)/n_split)
        for i in range(n_split):
            if i == n_split-1:
                #train index and test index
                yield (train_arr[i*train_each_num:], test_arr[i*test_each_num:])
            else:
                yield (train_arr[i*train_each_num:(i+1)*train_each_num], test_arr[i*test_each_num:(i+1)*test_each_num])

In [24]:
for train, test in split(uni, new_label,5):
    print(train.shape)
    print(test.shape)

NameError: name 'uni' is not defined

# model

In [38]:
NB = MultinomialNB()
pc = Perceptron()
svm = LinearSVC()
lr = LogisticRegression(solver='saga',max_iter=1000)
random_forest  = rf()
CNN = MLPClassifier()

In [16]:
KNN = knn(n_neighbors=3)

In [19]:
skf = StratifiedKFold(n_splits=5)

# Unigram

## HashingVectorizer

In [40]:
HashVec = HashingVectorizer(n_features = 500, 
                         ngram_range=(1,1))
Hash = HashVec.fit_transform(new_text)

In [41]:
for clf in [svm,lr]:
    acc = []
    for train_index, test_index in skf.split(Hash, new_label):
        x_train,x_test = Hash[train_index], Hash[test_index]
        y_train, y_test = new_label[train_index], new_label[test_index]
        clf.fit(x_train, y_train)
        acc.append(clf.score(x_test, y_test))
    acc = np.asarray(acc)
    print(acc.mean())

0.7241813490056019
0.726436031321512


## TfidVectorizer

In [44]:
TVec = TfidfVectorizer(ngram_range=(1,2), 
                       max_features=10000)
T = TVec.fit_transform(new_text)

In [45]:
for clf in [svm, lr]:
    acc = []
    for train_index, test_index in skf.split(T, new_label):
        x_train,x_test = T[train_index], T[test_index]
        y_train, y_test = new_label[train_index], new_label[test_index]
        clf.fit(x_train, y_train)
        acc.append(clf.score(x_test, y_test))
    acc = np.asarray(acc)
    print(acc.mean())

0.7900305260636258
0.783058785247783


## countvectorizer

In [46]:
UniVec = CountVectorizer(max_features = 9000, ngram_range = (1,2))
uni = UniVec.fit_transform(new_text)

In [34]:
#UniVec.vocabulary_

In [47]:
uni = np.asarray(uni.todense())

In [36]:
#test where to see the data format
print(uni.shape)
#uni[122]

(14640, 9000)


In [37]:
new_label

array([0, 1, 0, ..., 0, 2, 0])

# Confusion Matrix for SVM and LR based on Unigram

## SVM confusion matrix

In [22]:
#create confusion matrix 
svm_confusion_matrix = np.zeros((3,3))
#'neutral' 0
#'positive' 1
#'negative' 2
'''
x coordinate would be predicted labels
y coordinate would be true labels
'''

for clf in [svm]:
    for train_index, test_index in skf.split(uni, new_label):
        x_train,x_test = uni[train_index], uni[test_index]
        y_train, y_test = new_label[train_index], new_label[test_index]
        clf.fit(x_train, y_train)
        pred = clf.predict(x_test)
        for index, x_sample in enumerate(x_test):
            true_label = y_test[index]
            pred_label = pred[index]
            svm_confusion_matrix[true_label, pred_label] += 1

In [23]:
svm_confusion = np.zeros_like(svm_confusion_matrix)
for index, total in enumerate(np.sum(svm_confusion_matrix, axis=1)):
    svm_confusion[index] = svm_confusion_matrix[index]/total

In [24]:
svm_confusion

array([[0.53404324, 0.11648919, 0.34946757],
       [0.14515446, 0.61362675, 0.24121879],
       [0.08869035, 0.0343212 , 0.87698845]])

## Logistic regression confusion matrix

In [25]:
lr_confusion_matrix = np.zeros((3,3))
for clf in [lr]:
    for train_index, test_index in skf.split(uni, new_label):
        x_train,x_test = uni[train_index], uni[test_index]
        y_train, y_test = new_label[train_index], new_label[test_index]
        clf.fit(x_train, y_train)
        pred = clf.predict(x_test)
        for index, x_sample in enumerate(x_test):
            true_label = y_test[index]
            pred_label = pred[index]
            lr_confusion_matrix[true_label, pred_label] += 1

In [26]:
lr_confusion = np.zeros_like(lr_confusion_matrix)
for index, total in enumerate(np.sum(lr_confusion_matrix, axis=1)):
    lr_confusion[index] = lr_confusion_matrix[index]/total

In [27]:
lr_confusion

array([[0.53662472, 0.10487254, 0.35850274],
       [0.14261532, 0.60854846, 0.24883623],
       [0.08498584, 0.029854  , 0.88516017]])

# Unigram Models' Performance

In [ ]:
for clf in [lr]:
    acc = []
    for train_index, test_index in skf.split(uni, new_label):
        x_train,x_test = uni[train_index], uni[test_index]
        y_train, y_test = new_label[train_index], new_label[test_index]
        clf.fit(x_train, y_train)
        acc.append(clf.score(x_test, y_test))
    acc = np.asarray(acc)
    print(acc.mean())

In [59]:
#use my split()
'''
for clf in [ NB,pc, svm, lr]:
    acc = []
    for train_index, test_index in split(uni, new_label,5):
        x_train,x_test = uni[train_index], uni[test_index]
        y_train, y_test = new_label[train_index], new_label[test_index]
        clf.fit(x_train, y_train)
        acc.append(clf.score(x_test, y_test))
    acc = np.asarray(acc)
    print(acc.mean())
'''

'\nfor clf in [ NB,pc, svm, lr]:\n    acc = []\n    for train_index, test_index in split(uni, new_label,5):\n        x_train,x_test = uni[train_index], uni[test_index]\n        y_train, y_test = new_label[train_index], new_label[test_index]\n        clf.fit(x_train, y_train)\n        acc.append(clf.score(x_test, y_test))\n    acc = np.asarray(acc)\n    print(acc.mean())\n'

In [60]:
for clf in [KNN,random_forest]:
    acc = []
    for train_index, test_index in skf.split(uni, new_label):
        x_train,x_test = uni[train_index], uni[test_index]
        y_train, y_test = new_label[train_index], new_label[test_index]
        clf.fit(x_train, y_train)
        acc.append(clf.score(x_test, y_test))
    acc = np.asarray(acc)
    print(acc.mean())

0.7343598569265384


In [45]:
for clf in [CNN]:
    acc = []
    for train_index, test_index in skf.split(uni, new_label):
        x_train,x_test = uni[train_index], uni[test_index]
        y_train, y_test = new_label[train_index], new_label[test_index]
        clf.fit(x_train, y_train)
        acc.append(clf.score(x_test, y_test))
    acc = np.asarray(acc)
    print(acc.mean())

0.7445391110264828


# Bigram

In [47]:
BiVec = CountVectorizer(max_features = 500, ngram_range = (2,2))
Bi = BiVec.fit_transform(new_text)
skf = StratifiedKFold(n_splits=5)

In [48]:
for clf in [NB, pc, svm, lr, KNN, random_forest]:
    acc = []
    for train_index, test_index in skf.split(Bi, new_label):
        x_train,x_test = Bi[train_index], Bi[test_index]
        y_train, y_test = new_label[train_index], new_label[test_index]
        clf.fit(x_train, y_train)
        acc.append(clf.score(x_test, y_test))
    acc = np.asarray(acc)
    print(acc.mean())

0.6745232666005136
0.6002042115824728


/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.6852469157471445
0.6909827316494558
0.5290298961422412
0.6498645585559396


In [49]:
for clf in [CNN]:
    acc = []
    for train_index, test_index in skf.split(Bi, new_label):
        x_train,x_test = Bi[train_index], Bi[test_index]
        y_train, y_test = new_label[train_index], new_label[test_index]
        clf.fit(x_train, y_train)
        acc.append(clf.score(x_test, y_test))
    acc = np.asarray(acc)
    print(acc.mean())

0.6606575870982556


# Uni&Bigram

In [50]:
MixVec = CountVectorizer(max_features = 500, ngram_range = (1,2))
Mix = BiVec.fit_transform(new_text)
skf = StratifiedKFold(n_splits=5)

In [51]:
for clf in [NB, pc, svm, lr, KNN, random_forest]:
    acc = []
    for train_index, test_index in skf.split(Mix, new_label):
        x_train,x_test = Mix[train_index], Mix[test_index]
        y_train, y_test = new_label[train_index], new_label[test_index]
        clf.fit(x_train, y_train)
        acc.append(clf.score(x_test, y_test))
    acc = np.asarray(acc)
    print(acc.mean())

0.6745232666005136
0.6002042115824728


/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.6852469157471445
0.6909827316494558
0.5290298961422412
0.6521880371253316


In [52]:
for clf in [CNN]:
    acc = []
    for train_index, test_index in skf.split(Mix, new_label):
        x_train,x_test = Mix[train_index], Mix[test_index]
        y_train, y_test = new_label[train_index], new_label[test_index]
        clf.fit(x_train, y_train)
        acc.append(clf.score(x_test, y_test))
    acc = np.asarray(acc)
    print(acc.mean())

0.6651650148139308
